In [1]:
import spacy
import io
from google.colab import files
import pandas as pd



ModuleNotFoundError: No module named 'google.colab'

In [0]:
#for evaluation
install seqeval

SyntaxError: ignored

In [0]:
# uploading a csv file
uploaded = files.upload()
data = pd.read_csv(io.StringIO(uploaded['ner_dataset.csv'].decode('latin1'))).fillna(method="ffill")
data.tail(100)

Saving ner_dataset.csv to ner_dataset.csv


In [0]:
# spaCy model has limitation , so 100.000 examples are taken
data1 = data.loc[0:99999, :]
data1.tail()

,Sentence #,Word,POS,Tag
99995,Sentence: 4543,some,DT,O
99996,Sentence: 4543,seriously,RB,O
99997,Sentence: 4543,.,.,O
99998,Sentence: 4544,Demonstrators,NNS,O
99999,Sentence: 4544,chanting,VBG,O


In [0]:
initial_word_tag_tuple_list = list(zip(data1["Word"].values.tolist(), data1["Tag"].values.tolist()))
len(initial_word_tag_tuple_list)

In [0]:
initial_words = [x[0] for x in initial_word_tag_tuple_list]
print(len(initial_words))

In [0]:
#spaCy model requires text input
text = ' '.join(word for word in initial_words )
text

In [0]:
#spaCy model for English
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)
output_word_tag_lowered_tuple_list = []
for i in range(len(doc)):
  if doc[i].text in initial_words:
    output_word_tag_lowered_tuple_list.append((doc[i].text, doc[i].ent_iob_.lower() + '-' + doc[i].ent_type_.lower()))




101390


In [0]:
# getting aligned lists with prediction tags and true tags 
a = initial_word_tag_tuple_list
b = output_word_tag_lowered_tuple_list
y_true, y_pred = [], []
sentence = [x[0] for x in b]
for x in range(len(a)):
    if a[x][0] in sentence:
        y_true.append(a[x][1].lower())
        y_pred.append(b[sentence.index(a[x][0])][1].lower())
        sentence[x] = 'AnyRandomString'
print(len(y_true))
print(len(y_pred))

99136
99136


In [0]:

# tags are different, this snippet arranges to common tags
new_strings = []

for x in y_pred:
    if x == 'b-person':
      new_string = x.replace("b-person", "b-per")
      new_strings.append(new_string)
    elif x == 'i-person':
      new_string = x.replace("i-person", "i-per")
      new_strings.append(new_string)
    elif x == 'o-':
      new_string = x.replace("o-", "o")
      new_strings.append(new_string)
    elif x == 'b-loc':
      new_string = x.replace("b-loc", "b-geo")
      new_strings.append(new_string)
    elif x == 'i-loc':
      new_string = x.replace("i-loc", "i-geo")
      new_strings.append(new_string)
    elif x == 'b-event':
      new_string = x.replace("b-event", "b-eve")
      new_strings.append(new_string)
    elif x == 'i-event':
      new_string = x.replace("i-event", "i-eve")
      new_strings.append(new_string)
    elif x == 'b-date':
      new_string = x.replace("b-date", "b-tim")
      new_strings.append(new_string)
    elif x == 'i-date':
      new_string = x.replace("i-date", "i-tim")
      new_strings.append(new_string)
    elif x == 'b-time':
      new_string = x.replace("b-time", "b-tim")
      new_strings.append(new_string)
    elif x == 'i-time':
      new_string = x.replace("i-time", "i-tim")
      new_strings.append(new_string)
    elif x == 'b-gpe':
      new_strings.append('b-gpe')
    elif x == 'i-gpe':
      new_strings.append('i-gpe')
    elif x in ['i-norp','i-product','i-money','b-money','i-fac','b-quantity',
               'i-work_of_art','i-percent','b-law','i-quantity',
               'b-norp', 'b-product','b-work_of_art','i-cardinal',
               'b-cardinal', 'b-percent', 'b-language', 'b-fac', 'i-law','b-ordinal']:
      j = x
      new_string = x.replace(j,'o')
      new_strings.append(new_string)
    else:
      new_strings.append(x)

y_pred = new_strings
print(set(y_pred))
  

new_strings = []
for x in y_true:
  if x in ['b-art', 'i-nat','b-nat','i-art']:
    j = x
    new_string = x.replace(j, 'o')
    new_strings.append(new_string)
  else:

    new_strings.append(x)
y_true = new_strings
print(set(y_true))




{'b-org', 'b-eve', 'b-geo', 'i-per', 'b-per', 'i-geo', 'o', 'b-gpe', 'i-gpe', 'i-org', 'b-tim', 'i-tim', 'i-eve'}
{'b-org', 'b-eve', 'b-geo', 'i-per', 'b-per', 'i-geo', 'o', 'b-gpe', 'i-gpe', 'i-org', 'b-tim', 'i-tim', 'i-eve'}


In [0]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
 

print("Accuracy = {}".format(accuracy_score(y_true,y_pred)))
print("F1 = {}".format(f1_score(y_true,y_pred)))
print(classification_report(y_true,y_pred))

0.8826057133634603
0.4268780634190257
           precision    recall  f1-score   support

        o       0.54      0.53      0.53      9868
      org       0.46      0.44      0.45      1817
      geo       0.64      0.05      0.09      3283
      gpe       0.09      0.18      0.12      1738
      per       0.44      0.47      0.45      1658
      tim       0.48      0.69      0.57      1807
      eve       0.17      0.33      0.23        52

micro avg       0.43      0.42      0.43     20223
macro avg       0.50      0.42      0.41     20223

